In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-create-folds/train_folds.csv
/kaggle/input/icr-create-folds/__results__.html
/kaggle/input/icr-create-folds/__resultx__.html
/kaggle/input/icr-create-folds/__notebook__.ipynb
/kaggle/input/icr-create-folds/__output__.json
/kaggle/input/icr-create-folds/custom.css
/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv
/kaggle/input/ml-stratifiers/ml_stratifiers.py


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from catboost import Pool, CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('/kaggle/input/ml-stratifiers')

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ml_stratifiers import MultilabelStratifiedKFold

In [4]:
Train             = pd.read_csv('/kaggle/input/icr-create-folds/train_folds.csv')
Test              = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks            = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

In [5]:
Train['kfold'] = -1

kf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=Train, y=greeks.iloc[:,1:-1])):
    Train.loc[valid_indicies, "kfold"] = fold

In [6]:
Train['EJ'] = Train['EJ'].replace({'A': 0, 'B': 1})
Test['EJ']  = Test['EJ'].replace({'A': 0, 'B': 1})

In [7]:
Train.drop(columns=['BC', 'CL'], inplace=True)
Test.drop(columns=['BC', 'CL'], inplace=True)

In [8]:
features = Train.columns[1:-2]
label    = Train.columns[-2]

In [9]:
def balance_logloss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred / np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    
    logloss = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / (w0+w1)
    
    return logloss

In [10]:
final_valid_predictions = {}
final_test_predictions = []
s  = []
bs = []

for k in range(5):
    print('------------------ Fold: '+str(k))
    train     = Train[Train['kfold'] !=k].reset_index(drop=True)
    val       = Train[Train['kfold'] ==k].reset_index(drop=True)
    valid_ids = val.Id.values.tolist()

    train_dataset = Pool(data=train[features], label=train[label], cat_features=["EJ"] )
    eval_dataset  = Pool(data=val[features], label=val[label], cat_features=["EJ"])
    params = {
       "iterations": 10000,
        "verbose": 1000,
        "learning_rate": 0.003,
        "depth": 4,
        'auto_class_weights':'Balanced',
        'loss_function':'MultiClass',
        'eval_metric':'MultiClass:use_weights=False',
    }
    model = CatBoostClassifier(**params)
    model.fit(train_dataset, eval_set=eval_dataset, use_best_model=True)

    preds_valid = model.predict_proba(val[features])
    preds_test  = model.predict_proba(Test[features])
    final_test_predictions.append(preds_test)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    logloss  = log_loss(val[label], preds_valid)
    blogloss = balance_logloss(val[label], preds_valid)

    s.append(logloss)
    bs.append(blogloss)

    print(k, logloss, blogloss)
print('Log loss:')
print(s)
print(np.mean(s), np.std(s))  
print('Balance Log loss:')
print(bs)
print(np.mean(bs), np.std(bs))

------------------ Fold: 0
0:	learn: 0.6921696	test: 0.6922713	best: 0.6922713 (0)	total: 59.7ms	remaining: 9m 57s
1000:	learn: 0.2914837	test: 0.3332011	best: 0.3332011 (1000)	total: 3.62s	remaining: 32.5s
2000:	learn: 0.1884657	test: 0.2587154	best: 0.2587154 (2000)	total: 7.24s	remaining: 29s
3000:	learn: 0.1249975	test: 0.2219806	best: 0.2219806 (3000)	total: 10.8s	remaining: 25.3s
4000:	learn: 0.0882288	test: 0.2057668	best: 0.2057668 (4000)	total: 14.5s	remaining: 21.7s
5000:	learn: 0.0663189	test: 0.1971441	best: 0.1971441 (5000)	total: 18.1s	remaining: 18.1s
6000:	learn: 0.0523241	test: 0.1928064	best: 0.1928003 (5925)	total: 22s	remaining: 14.7s
7000:	learn: 0.0422477	test: 0.1904998	best: 0.1904944 (6999)	total: 25.6s	remaining: 11s
8000:	learn: 0.0349772	test: 0.1894683	best: 0.1893995 (7784)	total: 29.2s	remaining: 7.29s
9000:	learn: 0.0296379	test: 0.1885892	best: 0.1884775 (8944)	total: 32.8s	remaining: 3.63s
9999:	learn: 0.0255230	test: 0.1881262	best: 0.1881251 (9992)	t

In [11]:
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ['Id', 'class_0', 'class_1']
final_valid_predictions.to_csv(r"oof.csv", index=False)

final_test_predictions = (final_test_predictions[0] + final_test_predictions[1] + final_test_predictions[2] + final_test_predictions[3] + final_test_predictions[4])/5
test_dict = {}
test_dict.update(dict(zip(Test.Id.values.tolist(), final_test_predictions)))
submission = pd.DataFrame.from_dict(test_dict, orient="index").reset_index()
submission.columns = ['Id', 'class_0', 'class_1']                       

submission.to_csv(r"submission.csv", index=False)